# MEET THE NET
## C# 7.0

### Pattern matching

In [8]:
using System.Runtime.CompilerServices;

enum Operation { SystemTest, Start, Stop, Reset }
class State 
{ 
    public static State CreateState([CallerMemberName]string message = "") 
    { 
        Console.WriteLine(message);
        return new State();
    }
}

State PerformOperation(Operation command) =>
   command switch
   {
       Operation.SystemTest => RunDiagnostics(),
       Operation.Start => StartSystem(),
       Operation.Stop => StopSystem(),
       Operation.Reset => ResetToReady(),
       _ => throw new ArgumentException("Invalid enum value for command", nameof(command)),
   };
State PerformOperation(string command) =>
   command switch
   {
       "SystemTest" => RunDiagnostics(),
       "Start" => StartSystem(),
       "Stop" => StopSystem(),
       "Reset" => ResetToReady(),
       _ => throw new ArgumentException("Invalid string value for command", nameof(command)),
   };

State PerformOperation(string command, int seconds) =>
   command switch
   {
       "SystemTest" when seconds < 10 => RunDiagnostics(),
       "Start" when seconds >= 10 && seconds <= 20 => StartSystem(),
       "Stop" => StopSystem(),
       "Reset" => ResetToReady(),
       _ => throw new ArgumentException("Invalid string value for command", nameof(command)),
   };

State RunDiagnostics() => State.CreateState();
State StartSystem() => State.CreateState();
State StopSystem() => State.CreateState();
State ResetToReady() => State.CreateState();
PerformOperation("Start");

Error: System.ArgumentException: Invalid string value for command (Parameter 'command')
   at Submission#9.PerformOperation(String command, Int32 seconds)
   at Submission#9.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
string WaterState(int tempInFahrenheit) =>
    tempInFahrenheit switch
    {
        (> 32) and (< 212) => "liquid",
        < 32 => "solid",
        > 212 => "gas",
        32 => "solid/liquid transition",
        212 => "liquid / gas transition",
    };

string WaterState2(int tempInFahrenheit) =>
    tempInFahrenheit switch
    {
        < 32 => "solid",
        32 => "solid/liquid transition",
        < 212 => "liquid",
        212 => "liquid / gas transition",
        _ => "gas",
};

Console.WriteLine(WaterState(10));
Console.WriteLine(WaterState2(10));

In [11]:
class Order
{
    public int Items { get; set; }
    public decimal Cost { get; set; }

    public void Deconstruct(out int items, out decimal cost)
    {
        items = Items;
        cost = Cost;
    }
}

decimal CalculateDiscount(Order order) =>
    order switch
    {
        { Items: > 10, Cost: > 1000.00m } => 0.10m,
        { Items: > 5, Cost: > 500.00m } => 0.05m,
        { Cost: > 250.00m } => 0.02m,
        null => throw new ArgumentNullException(nameof(order), "Can't calculate discount on null order"),
        var someObject => someObject.Cost,
    };

decimal CalculateDiscount2(Order order) =>
    order switch
    {
        ( > 10,  > 1000.00m) => 0.10m,
        ( > 5, > 50.00m) => 0.05m,
        { Cost: > 250.00m } => 0.02m,
        null => throw new ArgumentNullException(nameof(order), "Can't calculate discount on null order"),
        var someObject => someObject.Cost,
    };

Console.WriteLine(CalculateDiscount(new Order { Items = 10, Cost = 20m }));
Console.WriteLine(CalculateDiscount2(new Order { Items = 10, Cost = 20m }));

20
20


In [16]:
List<string[]> stringList = new List<string[]>()
{
    new string[] { "04-01-2020", "DEPOSIT"   , "Initial deposit",                     "2250.00" },
    new string[] { "04-15-2020", "DEPOSIT"   , "Refund"         ,                     "125.65"  },
    new string[] { "04-18-2020", "DEPOSIT"   , "Paycheck"       ,                     "825.65"  },
    new string[] { "04-22-2020", "WITHDRAWAL", "Debit"          , "Groceries",        "255.73"  },
    new string[] { "05-01-2020", "WITHDRAWAL", "#1102"          , "Rent"     , "apt", "2100.00" },
    new string[] { "05-02-2020", "INTEREST"  ,                                        "0.65"    },
    new string[] { "05-07-2020", "WITHDRAWAL", "Debit"          , "Movies"   ,        "12.57"   },
    new string[] { "04-15-2020", "FEE"       ,                                        "5.55"    },
};

decimal balance = 0m;
foreach (string[] transaction in stringList)
{
    balance += transaction switch
    {
        [_, "DEPOSIT"   , _ , var amount] => decimal.Parse(amount),
        [_, "WITHDRAWAL", .., var amount] => -decimal.Parse(amount),
        [_, "INTEREST"  ,     var amount] => decimal.Parse(amount),
        [_, "FEE"       ,     var fee]    => -decimal.Parse(fee),
        _                                 => throw new InvalidOperationException($"Record {string.Join(", ", transaction)} is not in the expected format!"),
    };
    Console.WriteLine($"Record: {string.Join(", ", transaction)}, New balance: {balance:C}");
}

Record: 04-01-2020, DEPOSIT, Initial deposit, 2250.00, New balance: $2,250.00
Record: 04-15-2020, DEPOSIT, Refund, 125.65, New balance: $2,375.65
Record: 04-18-2020, DEPOSIT, Paycheck, 825.65, New balance: $3,201.30
Record: 04-22-2020, WITHDRAWAL, Debit, Groceries, 255.73, New balance: $2,945.57
Record: 05-01-2020, WITHDRAWAL, #1102, Rent, apt, 2100.00, New balance: $845.57
Record: 05-02-2020, INTEREST, 0.65, New balance: $846.22
Record: 05-07-2020, WITHDRAWAL, Debit, Movies, 12.57, New balance: $833.65
Record: 04-15-2020, FEE, 5.55, New balance: $828.10


## C# 9.0 (NET5)
### Record types

#### Kotlin example
`data class User(val name: String, val age: Int)`

In [8]:
public record SmallPerson(string FirstName, string LastName);
public record BigPerson
{
    public string FirstName { get; init; } = default!;
    public string LastName { get; init; } = default!;
}